## Линейные методы. Vowpal Wabbit.

Vowpal Wabbit on GitHub: https://github.com/JohnLangford/vowpal_wabbit

Vowpal Wabbit Tutorial: https://github.com/JohnLangford/vowpal_wabbit/wiki/Tutorial

In [1]:
!apt-get install vowpal-wabbit

Reading package lists... Done
Building dependency tree       
Reading state information... Done
vowpal-wabbit is already the newest version (8.5.0.dfsg1-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [12]:
!wget https://www.dropbox.com/s/crld672bipr0n05/train-sample.csv?dl=0

--2019-01-09 09:29:05--  https://www.dropbox.com/s/crld672bipr0n05/train-sample.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:601a:1::a27d:701
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/crld672bipr0n05/train-sample.csv [following]
--2019-01-09 09:29:05--  https://www.dropbox.com/s/raw/crld672bipr0n05/train-sample.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9f3bcb25b1aa2ca12e4d302228.dl.dropboxusercontent.com/cd/0/inline/AZFtkMZg6sY4RvvT4D5zxt4jT91uFxj40qehS3gKcJq5hr0XP-IXFfFeTQmYyusl5QcjUOSVabgT-I6N6cbte-vQw78uE6AfO26OmDr3D85sFfi3k8UhmZ84NJvQe9DPY5ZYHjF6pYusW7SZ7fLXR-LVOkry1MuvAMusEII-FHan-HV91rwRM7H-LoNYsrc92y4/file [following]
--2019-01-09 09:29:05--  https://uc9f3bcb25b1aa2ca12e4d302228.dl.dropboxusercontent.com/cd/0/inline/AZFtkMZg6sY4RvvT4D5zxt4jT91uFxj40q

In [13]:
!ls

 drive	       train-sample.csv        'train-sample.rar?dl=0'
 sample_data  'train-sample.csv?dl=0'


In [0]:
import pandas as pd
import numpy as np

In [0]:
train_path = 'train-sample.csv?dl=0'

In [15]:
data = pd.read_csv(train_path)
data.head()
print(len(data))

140272


In [16]:
print(data.OpenStatus[10])
print(data.BodyMarkdown[10])

open
i have two dimensions, first (width, height) and second(width1, height1). how can i retrieve a Point(x,y) from dimensions???


In [0]:
data_train = data.iloc[:50000, :]
data_test = data.iloc[70000:, :]

In [0]:
import re

def save_to_vw(data, fname):
    with open(fname, 'w') as fout:
        for _, row in data.iterrows():
            text = filter(lambda x: len(x) > 1, re.split("[^a-z]",
                                    row.BodyMarkdown.lower()))
            text = ' '.join(text)
            if row.OpenStatus == "open":
                target = 1
            else:
                target = -1
            fout.write('{0} |n 0:{1} {2} |t {3}\n'.format(target, 
                                        row.ReputationAtPostCreation,
                                        row.Tag1,
                                        text))

In [0]:
save_to_vw(data_train, 'train.vw')
save_to_vw(data_test, 'test.vw')

In [20]:
!ls

 drive	       test.vw		 'train-sample.csv?dl=0'   train.vw
 sample_data   train-sample.csv  'train-sample.rar?dl=0'


In [21]:
!head -n 2 train.vw

1 |n 0:1 mongodb |t am building corpus of indexed sentences in different languages have collection of languages which have both an objectid and the iso code as key is it better to use reference to the language collection or store key like en or fr suppose it compromise between ease of referencing the language object in that collection speed in doing queries where the sentence has certain language the size of the data on disk any best practices that should know of
1 |n 0:192 dom |t create xml document with jaxp and search way to insert the schemalocation at the moment my application produces xml version encoding utf root root but need xml version encoding utf root xmlns namespaceurl xmlns xs http www org xmlschema instance xs schemalocation namespaceurl pathtomyschema xsd root my code streamresult result new streamresult writer document doc getdocument transformer trans transfac newtransformer trans setoutputproperty outputkeys indent yes trans setoutputproperty outputkeys method xml tr

In [32]:
!vw -d train.vw -c -k -f model.vw --passes 10 --link logistic

final_regressor = model.vw
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.vw.cache
Reading datafile = train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0   1.0000   0.5000       81
0.500000 0.000000            2            2.0   1.0000   0.7311       98
0.906326 1.312652            4            4.0  -1.0000   0.6051      110
0.821724 0.737121            8            8.0  -1.0000   0.6143       38
0.875065 0.928406           16           16.0   1.0000   0.6741      136
0.906826 0.938586           32           32.0   1.0000   0.6249       93
0.996030 1.085234           64           64.0   1.0000   0.6714       71
0.977266 0.958502          128          128.0  -1.0000   0.4202       24
0.990424 1.003582          256          256.0  -1.0000   0.5678      

In [33]:
!vw -d test.vw -i model.vw -t -p pred.txt

only testing
predictions = pred.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0   1.0000   0.7311      187
0.000000 0.000000            2            2.0   1.0000   0.7311      126
1.000000 2.000000            4            4.0  -1.0000   0.7311      516
0.650943 0.301887            8            8.0   1.0000   0.6917      117
0.710289 0.769635           16           16.0   1.0000   0.5795      155
0.613725 0.517161           32           32.0   1.0000   0.4811       48
0.827658 1.041591           64           64.0  -1.0000   0.7311      226
0.809121 0.790584          128          128.0   1.0000   0.5220       86
0.811053 0.812985          256          256.0  -1.0000   0.5069       83
0.832518 0.853984          512    

In [34]:
!head -n 10 pred.txt

0.731059
0.731059
0.268941
0.731059
0.520479
0.731059
0.268941
0.691661
0.462321
0.344321


In [35]:
from sklearn.metrics import roc_auc_score

def calc_vw_qual():
    preds = pd.read_csv('pred.txt', header=None).iloc[:, 0].values
    target = data_test.OpenStatus.values
    T = []
    for t in target:
        if t == 'open':
            T.append(1.)
        else:
            T.append(-1.)
    print(roc_auc_score(T, preds))
    
calc_vw_qual()

0.7749402873566568


In [36]:
!vw -d train.vw -c -k -f model.vw --passes 10 -l 0.1 --link logistic
!vw -d test.vw -i model.vw -t -p pred.txt
print('\n\n\n')
calc_vw_qual()

final_regressor = model.vw
Num weight bits = 18
learning rate = 0.1
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.vw.cache
Reading datafile = train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0   1.0000   0.5000       81
0.500000 0.000000            2            2.0   1.0000   0.7311       98
0.812846 1.125692            4            4.0  -1.0000   0.5555      110
0.769143 0.725440            8            8.0  -1.0000   0.5734       38
0.845550 0.921957           16           16.0   1.0000   0.6202      136
0.812974 0.780397           32           32.0   1.0000   0.6439       93
0.899824 0.986675           64           64.0   1.0000   0.5865       71
0.894960 0.890097          128          128.0  -1.0000   0.4753       24
0.893272 0.891584          256          256.0  -1.0000   0.5065      

n-граммы (n=2) - индикаторы того, что два слова встретились рядом. Из "мама мыла раму" получаем биграммы "мама мыла" и "мыла раму".

In [37]:
!vw -d train.vw -c -k -f model.vw --passes 10 -l 0.1 --ngram t2 --link logistic
!vw -d test.vw -i model.vw -t -p pred.txt
print('\n\n\n')
calc_vw_qual()

Generating 2-grams for t namespaces.
final_regressor = model.vw
Num weight bits = 18
learning rate = 0.1
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.vw.cache
Reading datafile = train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0   1.0000   0.5000      158
0.501991 0.003982            2            2.0   1.0000   0.7185      192
0.769028 1.036065            4            4.0  -1.0000   0.5258      216
0.788365 0.807701            8            8.0  -1.0000   0.5554       72
0.853561 0.918757           16           16.0   1.0000   0.5926      268
0.815435 0.777309           32           32.0   1.0000   0.6290      182
0.899736 0.984037           64           64.0   1.0000   0.5476      138
0.899076 0.898416          128          128.0  -1.0000   0.4850       44
0.900915 0.902754          256  

k-skip-n-граммы - как n-граммы, но разрешаем словам быть отдаленными друг от друга не больше, чем на k

In [38]:
!vw -d train.vw -c -k -f model.vw --passes 10 -l 0.1 --ngram t2 --skips t2 --link logistic
!vw -d test.vw -i model.vw -t -p pred.txt
print('\n\n\n')
calc_vw_qual()

Generating 2-grams for t namespaces.
Generating 2-skips for t namespaces.
final_regressor = model.vw
Num weight bits = 18
learning rate = 0.1
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.vw.cache
Reading datafile = train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0   1.0000   0.5000      309
0.598903 0.197807            2            2.0   1.0000   0.6354      377
0.813193 1.027482            4            4.0  -1.0000   0.5188      425
0.827788 0.842383            8            8.0  -1.0000   0.5387      137
0.878166 0.928544           16           16.0   1.0000   0.5696      529
0.833454 0.788743           32           32.0   1.0000   0.6141      357
0.919097 1.004739           64           64.0   1.0000   0.5475      269
0.925038 0.930980          128          128.0  -1.0000   0.4903     

In [39]:
!vw -d train.vw -c -k -f model.vw --passes 10 -l 0.1 --ngram t2 -b 28 --link logistic
!vw -d test.vw -i model.vw -t -p pred.txt
print('\n\n\n')
calc_vw_qual()

Generating 2-grams for t namespaces.
final_regressor = model.vw
Num weight bits = 28
learning rate = 0.1
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
tcmalloc: large alloc 4294967296 bytes == 0x560b42dd2000 @  0x7fe460ed0001 0x7fe460a6cb5f 0x7fe460a7aa21 0x7fe460b1de00 0x7fe460b0bbe3 0x7fe460b13395 0x7fe460b13c44 0x560b40ba7237 0x560b40ba6a8b 0x7fe46008bb97 0x560b40ba705a
creating cache_file = train.vw.cache
Reading datafile = train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0   1.0000   0.5000      158
0.501991 0.003982            2            2.0   1.0000   0.7185      192
0.768823 1.035654            4            4.0  -1.0000   0.5257      216
0.788250 0.807676            8            8.0  -1.0000   0.5554       72
0.852059 0.915868           16           16.0   1.0000   0.5925      268
0.814879 0.777699         